In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

In [ ]:
def loadMovieNames():
    movieNames = {}
    # CHANGE THIS TO THE PATH TO YOUR u.ITEM FILE:
    with codecs.open("./ml-100k/u.ITEM", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [ ]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [ ]:
nameDict = spark.sparkContext.broadcast(loadMovieNames())

Create schema when reading u.data

In [ ]:
schema = StructType([ \
                     StructField("userID", IntegerType(), True), \
                     StructField("movieID", IntegerType(), True), \
                     StructField("rating", IntegerType(), True), \
                     StructField("timestamp", LongType(), True)])

Load up movie data as dataframe

In [ ]:
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("./ml-100k/u.data")

In [ ]:
movieCounts = moviesDF.groupBy("movieID").count()

Create a user-defined function to look up movie names from our broadcasted dictionary

In [ ]:
def lookupName(movieID):
    return nameDict.value[movieID]

In [ ]:
lookupNameUDF = func.udf(lookupName)

Add a movieTitle column using our new udf

In [ ]:
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

Sort the results

In [ ]:
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

Grab the top 10

In [ ]:
sortedMoviesWithNames.show(10, False)

Stop the session

In [ ]:
spark.stop()